In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import tools

from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize

In [2]:
from pandas import set_option
set_option("display.max_rows", 10)
pd.options.mode.chained_assignment = None

filename = '../training_data.csv'
df = pd.read_csv(filename)
df

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,3,A1 SH,SHRIMPLIN,2793.0,77.450,0.664,9.900,11.915,4.600,1,1.000
1,3,A1 SH,SHRIMPLIN,2793.5,78.260,0.661,14.200,12.565,4.100,1,0.979
2,3,A1 SH,SHRIMPLIN,2794.0,79.050,0.658,14.800,13.050,3.600,1,0.957
3,3,A1 SH,SHRIMPLIN,2794.5,86.100,0.655,13.900,13.115,3.500,1,0.936
4,3,A1 SH,SHRIMPLIN,2795.0,74.580,0.647,13.500,13.300,3.400,1,0.915
...,...,...,...,...,...,...,...,...,...,...,...
3227,5,C LM,CHURCHMAN BIBLE,3120.5,46.719,0.947,1.828,7.254,3.617,2,0.685
3228,5,C LM,CHURCHMAN BIBLE,3121.0,44.563,0.953,2.241,8.013,3.344,2,0.677
3229,5,C LM,CHURCHMAN BIBLE,3121.5,49.719,0.964,2.925,8.013,3.190,2,0.669
3230,5,C LM,CHURCHMAN BIBLE,3122.0,51.469,0.965,3.083,7.708,3.152,2,0.661


In [3]:
df['Well Name'].unique()

array(['SHRIMPLIN', 'SHANKLE', 'LUKE G U', 'CROSS H CATTLE', 'NOLAN',
       'Recruit F9', 'NEWBY', 'CHURCHMAN BIBLE'], dtype=object)

In [4]:
df = df.set_index(['Well Name', 'Depth'], drop=False)
df

Facies Formation        Well Name   Depth      GR  \
Well Name       Depth                                                       
SHRIMPLIN       2793.0       3     A1 SH        SHRIMPLIN  2793.0  77.450   
                2793.5       3     A1 SH        SHRIMPLIN  2793.5  78.260   
                2794.0       3     A1 SH        SHRIMPLIN  2794.0  79.050   
                2794.5       3     A1 SH        SHRIMPLIN  2794.5  86.100   
                2795.0       3     A1 SH        SHRIMPLIN  2795.0  74.580   
...                        ...       ...              ...     ...     ...   
CHURCHMAN BIBLE 3120.5       5      C LM  CHURCHMAN BIBLE  3120.5  46.719   
                3121.0       5      C LM  CHURCHMAN BIBLE  3121.0  44.563   
                3121.5       5      C LM  CHURCHMAN BIBLE  3121.5  49.719   
                3122.0       5      C LM  CHURCHMAN BIBLE  3122.0  51.469   
                3122.5       5      C LM  CHURCHMAN BIBLE  3122.5  50.031   

                        ILD_log10  DeltaPHI   PHIND     PE  NM_M  RELPOS  
Well Name       Depth                                                     
SHRIMPLIN       2793.0      0.664     9.900  11.915  4.600     1   1.000  
                2793.5      0.661    14.200  12.565  4.100     1   0.979  
                2794.0      0.658    14.800  13.050  3.600     1   0.957  
                2794.5      0.655    13.900  13.115  3.500     1   0.936  
                2795.0      0.647    13.500  13.300  3.400     1   0.915  
...                           ...       ...     ...    ...   ...     ...  
CHURCHMAN BIBLE 3120.5      0.947     1.828   7.254  3.617     2   0.685  
                3121.0      0.953     2.241   8.013  3.344     2   0.677  
                3121.5      0.964     2.925   8.013  3.190     2   0.669  
                3122.0      0.965     3.083   7.708  3.152     2   0.661  
                3122.5      0.970     2.609   6.668  3.295     2   0.653  

[3232 rows x 11 columns]

In [8]:
rf9 = df.loc['Recruit F9']
rf9

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
Depth,,,,,,,,,,,
3018.0,9,C LM,Recruit F9,3018.0,32.780,0.723,1.200,13.110,5.100,2,0.186
3018.5,9,C LM,Recruit F9,3018.5,23.500,0.711,0.300,12.300,5.200,2,0.176
3019.0,9,C LM,Recruit F9,3019.0,28.330,0.706,0.500,11.790,5.300,2,0.167
3019.5,9,C LM,Recruit F9,3019.5,31.120,0.704,1.700,11.325,5.300,2,0.157
3020.0,9,C LM,Recruit F9,3020.0,28.680,0.709,1.900,11.165,5.400,2,0.147
...,...,...,...,...,...,...,...,...,...,...,...
3079.0,9,B5 LM,Recruit F9,3079.0,185.250,0.830,3.480,7.995,3.912,2,0.167
3079.5,9,B5 LM,Recruit F9,3079.5,222.500,0.852,4.614,6.959,3.557,2,0.150
3080.0,9,B5 LM,Recruit F9,3080.0,74.000,0.768,0.243,9.670,5.672,2,0.200


In [ ]:
df.loc['SHRIMPLIN']

In [ ]:
tools.make_facies_log_plot(df.loc['SHRIMPLIN'])

In [ ]:
features = df.drop(['NM_M','RELPOS','Depth','Well Name','Formation','Facies'], axis=1)
features

In [ ]:
X = features.sub(features.mean(axis=1), axis=0).values
y = df['Facies'].values

In [ ]:
model = TSNE(n_components=2, init='pca', method='exact', learning_rate=1000, random_state=0)
np.set_printoptions(suppress=True)
yy = model.fit_transform(X)
yy

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
plt.scatter(X[:,1], X[:,2], c=y, cmap=plt.get_cmap('Set1'))
X[:,0].shape
